In [1]:
%matplotlib inline
import pandas
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import normalize

In [2]:
data_url="http://goo.gl/vhm1eU"

In [3]:
names = ['prg', 'plas', 'pres','skin','test','mass','pedi', 'age','class']

df = pandas.read_csv(data_url, names=names)
df[:5]

,prg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
y_data=df['class']
#y_data=y_data.reshape(len(y_data),1)
y_data[:5]

0    1
1    0
2    1
3    0
4    1
Name: class, dtype: int64

In [5]:
x_data=df.ix[:,:-1]
norm_x= normalize(x_data, axis=0)
#norm_x=(x_data-x_data.min())/(x_data.max()-x_data.min())
norm_x[:5]

array([[ 0.04235974,  0.04270834,  0.03620428,  0.04857917,  0.        ,
         0.03679789,  0.03924821,  0.05117289],
       [ 0.00705996,  0.02452844,  0.03318725,  0.04025131,  0.        ,
         0.02913166,  0.02197149,  0.03172719],
       [ 0.05647966,  0.05280828,  0.03218158,  0.        ,  0.        ,
         0.02551759,  0.04206507,  0.03275065],
       [ 0.00705996,  0.02568272,  0.03318725,  0.03192345,  0.02420843,
         0.03077443,  0.01045367,  0.02149261],
       [ 0.        ,  0.03953407,  0.02011349,  0.04857917,  0.04326614,
         0.04720206,  0.14322153,  0.03377411]])

In [6]:
x_data=norm_x#.values
x1=np.ones(len(x_data))
x1=x1.reshape(len(x_data),1)
x_data=np.float32(np.concatenate((x1, x_data), axis=1))
x_data=np.transpose(x_data)
x_data[:5]

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.04235974,  0.00705996,  0.05647966, ...,  0.03529979,
         0.00705996,  0.00705996],
       [ 0.04270834,  0.02452844,  0.05280828, ...,  0.03491695,
         0.0363598 ,  0.026837  ],
       [ 0.03620427,  0.03318725,  0.03218158, ...,  0.03620427,
         0.03017023,  0.0351986 ],
       [ 0.04857917,  0.04025131,  0.        , ...,  0.03192345,
         0.        ,  0.04302726]], dtype=float32)

In [7]:
X=tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

W = tf.Variable(tf.random_uniform([1,len(x_data)], -1.0, 1.0))


In [8]:
h=tf.matmul(W,X)
hypothesis = tf.div(1., 1. + tf.exp(-h))
cost = -1 * tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))

In [9]:
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for step in range(10000000):
    sess.run(train, feed_dict={X: x_data, Y:y_data})
    if step % 100000 == 0:
        print (step, sess.run(cost, feed_dict={X: x_data, Y:y_data}),sess.run(W))
        

0 0.72422 [[ 0.20879316  0.38308755  0.52283001 -0.55434912  0.58335137  0.44947597
  -0.62164867 -0.27865943 -0.93770659]]
100000 0.564126 [[ -2.68022299  13.44687748  16.913517     0.40945342   2.3534112
    5.89563894   9.00977993   9.40341568   7.86028671]]
200000 0.539109 [[ -3.65516233  16.1175251   29.33483696  -0.26783764   1.80871129
    5.33218622  16.52249527  12.67670345  11.77918911]]
300000 0.523443 [[ -4.29422474  16.30084419  39.63007355  -1.44418454   1.32804143
    4.31292105  22.91154099  13.89923477  13.91910362]]
400000 0.512167 [[ -4.78008127  15.95717812  48.38736725  -2.8023932    1.01441789
    3.3502636   28.4509697   14.33115864  15.22558117]]
500000 0.503737 [[ -5.17767811  15.65501595  55.93929291  -4.21976042   0.82467365
    2.51066995  33.28853226  14.4615984   16.05209732]]
600000 0.497298 [[ -5.51206255  15.41596889  62.48389816  -5.6274538    0.71092778
    1.78184366  37.59082413  14.46552849  16.56170654]]
700000 0.492325 [[ -5.79861879  15.27264404

KeyboardInterrupt: 

In [10]:
w_list=sess.run(W)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [11]:
x_data=np.transpose(x_data)
w_list=w_list.T
x_data.dot(w_list)[:5]


array([[ 0.9146359 ],
       [-2.87844801],
       [ 1.3072145 ],
       [-3.04791832],
       [ 2.12373447]], dtype=float32)

In [12]:
y_data=y_data.reshape(len(y_data),1)
sum((sigmoid(x_data.dot(w_list))>0.5) ==y_data)/y_data.shape[0]

array([ 0.78385417])